<a href="https://colab.research.google.com/github/kanishka-rani-2005/Deep_Learning/blob/main/Next_Word_Generator/NextWordPredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
## DATA COLLECTION
import nltk
import pandas as pd
import numpy as np
nltk.download('gutenberg')
from nltk.corpus import gutenberg


[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [28]:
## load the data
data = gutenberg.raw('shakespeare-hamlet.txt')

with open('hamlet.txt','w') as file:
  file.write(data)

In [29]:
## Data Preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [30]:
## Load dataset
with open('hamlet.txt','r') as file:
  text=file.read().lower()

## Tokenize the text
tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])


In [31]:
total_words=len(tokenizer.word_index)+1 ## because of zero based indexing
total_words

4818

In [32]:
# tokenizer.word_counts

In [33]:
# tokenizer.word_docs

In [34]:
# tokenizer.word_index

In [35]:
# tokenizer.texts_to_sequences('hello i am kanishka ')

In [36]:
## Create input Sequence

input_sequences=[]

for line in text.split('\n'):
  token_list=tokenizer.texts_to_sequences([line])[0]
  for i in range(1,len(token_list)):
    n_gram_sequence=token_list[:i+1]
    input_sequences.append(n_gram_sequence)


In [37]:
# input_sequences

In [38]:
## Pad Sequence
max_sequence_len=max([len(x) for x in input_sequences])
max_sequence_len

14

In [39]:
input_sequences=np.array(pad_sequences(input_sequences,maxlen=max_sequence_len,padding='pre'))
input_sequences


array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]], dtype=int32)

In [40]:
## Create predictor and label
import tensorflow as tf

## x=all words except last and y=only last words
x,y=input_sequences[:,:-1],input_sequences[:,-1]


In [41]:
y

array([ 687,    4,   45, ..., 1047,    4,  193], dtype=int32)

In [42]:
y=tf.keras.utils.to_categorical(y,num_classes=total_words)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [43]:
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
print(X_train.shape,X_test.shape,y_test.shape,y_train.shape)

(20585, 13) (5147, 13) (5147, 4818) (20585, 4818)


In [44]:
## Train LSTM RNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [45]:
model=Sequential()
model.add(Embedding(total_words,100,input_length=max_sequence_len))
model.add(LSTM(150,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words,activation='softmax'))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [46]:
 model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [47]:
early_stopping = EarlyStopping(monitor='val_loss', patience=60)

In [48]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [49]:
history=model.fit(X_train,y_train,batch_size=32,epochs=100,callbacks=[early_stopping],validation_data=(X_test,y_test))


Epoch 1/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.0307 - loss: 7.1587 - val_accuracy: 0.0336 - val_loss: 6.6998
Epoch 2/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.0352 - loss: 6.4629 - val_accuracy: 0.0422 - val_loss: 6.7523
Epoch 3/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.0443 - loss: 6.3316 - val_accuracy: 0.0484 - val_loss: 6.7894
Epoch 4/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.0511 - loss: 6.1605 - val_accuracy: 0.0509 - val_loss: 6.7996
Epoch 5/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.0544 - loss: 6.0312 - val_accuracy: 0.0556 - val_loss: 6.8424
Epoch 6/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.0590 - loss: 5.8942 - val_accuracy: 0.0622 - val_loss: 6.8840
Epoch 7/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.0730 - loss: 5.7193 - val_accuracy: 0.0614 - val_loss: 6.9243
Epoch 8/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.0813 - loss: 5.5858 

In [50]:
def predict_next_word(model,tokenizer,text,maxlen):
  token_list=tokenizer.texts_to_sequences([text])[0]

  if(len(token_list))>=maxlen:
    token_list=token_list[-(maxlen-1):]
  token_list=pad_sequences([token_list],maxlen=maxlen-1,padding='pre')
  predicted=model.predict(token_list,verbose=1)
  predicted_word_index=np.argmax(predicted,axis=1)

  for word ,index in tokenizer.word_index.items():
    if index==predicted_word_index:
      return word
  return None

In [51]:
input_text='to be or not to be'

print(f"Input text is :{input_text}")
maxlen=model.input_shape[1]+1
next_word=predict_next_word(model,tokenizer,input_text,maxlen)
print(f"Next word is :{next_word}")


Input text is :to be or not to be
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
Next word is :more


In [52]:
model.save('next_word_predictor.h5')

In [53]:
import pickle
with open('tokenizer.pkl','wb') as file:
  pickle.dump(tokenizer,file)